In [10]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from scipy.signal.windows import gaussian
from scipy.ndimage import convolve1d
import yfinance as yf
import matplotlib.pyplot as plt


# Init all data
df1 = pd.read_excel('xauusd.xlsx', usecols=['Date', 'Open', 'Price'])
df2 = pd.read_csv('dxy.csv', usecols=['Date', 'Open'])
df3 = pd.read_csv('r.csv', usecols=['Date', 'Fed Rate'])
df4 = yf.download('^GSPC', start='1930-01-01') #S&P 500
df5 = pd.read_csv('cpi.csv', usecols=['Date', 'CPI'])

# Use 'Date' column as datetime type
df4.reset_index(inplace=True)
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])
df3['Date'] = pd.to_datetime(df3['Date'], dayfirst=True)
df4['Date'] = pd.to_datetime(df4['Date'])
df5['Date'] = pd.to_datetime(df5['Date'])

# Clean all data
df5.set_index('Date', inplace=True)
daily_dates = pd.date_range(start=df5.index[0], end=df5.index[-1] + pd.offsets.MonthEnd(), freq='D')
df5 = df5.reindex(index=daily_dates).ffill()
df5.reset_index(inplace=True)
df5.columns = ['Date', 'CPI']

merged_df = pd.merge(df2, df3, on='Date', how='outer')
merged_df = pd.merge(merged_df, df4, on='Date', how='outer')
merged_df = pd.merge(merged_df, df5, on='Date', how='outer')
merged_df = pd.merge(merged_df, df1, on='Date', how='right')
month = merged_df['Date'].dt.month
day = merged_df['Date'].dt.day

merged_df['Close'] = merged_df['Price'].replace(',', '', regex=True).astype(float)
merged_df['Open'] = merged_df['Open'].replace(',', '', regex=True).astype(float)
merged_df['DXY'] = merged_df['Open_x']
merged_df['SPX'] = merged_df['Open_y']
merged_df['t'] = (month - 1) * 31 + day
merged_df = merged_df.drop(['High', 'Low', 'Adj Close', 'Volume'], axis=1)
raw = merged_df.drop(['Open_y', 'Price', 'Open_x'], axis=1)

start_date = '1986-05-14' # day with diff in close and open
end_date = '2024-01-01'
raw = raw[(raw['Date'] > start_date) & (raw['Date'] <= end_date)]
features = ['Open', 'Close','Fed Rate', 'CPI', 'DXY', 't']
for feature in features:
    nan_count = raw[feature].isna().sum()
    raw.ffill(inplace=True)
    
print(raw)

[*********************100%%**********************]  1 of 1 completed

            Date  Fed Rate    Close      CPI     Open         DXY  \
1625  1986-05-15      7.00   339.57  110.500   340.81  111.500000   
1626  1986-05-16      6.80   340.81  110.500   341.13  112.559998   
1627  1986-05-19      6.89   338.21  110.500   340.40  113.279999   
1628  1986-05-20      6.86   336.03  110.500   337.90  112.919998   
1629  1986-05-21      6.96   335.93  110.500   336.65  113.529999   
...          ...       ...      ...      ...      ...         ...   
11407 2023-12-26      5.33  2067.20  306.746  2055.73  101.650002   
11408 2023-12-27      5.33  2077.00  306.746  2066.70  101.550003   
11409 2023-12-28      5.33  2066.04  306.746  2077.34  100.870003   
11410 2023-12-29      5.33  2062.59  306.746  2065.15  101.199997   
11411 2024-01-01      5.33  2063.80  308.417  2064.24  101.199997   

               SPX    t  
1625    237.539993  139  
1626    234.429993  140  
1627    232.759995  143  
1628    233.199997  144  
1629    236.110001  145  
...            

In [11]:
raw.to_excel('raw.xlsx') 

In [9]:
# XAUUSD merging
df1 = pd.read_csv('xauusd1.csv', usecols=['Date', 'Open', 'Price'])
df11 = pd.read_csv('xauusd4.csv', usecols=['Date', 'Open', 'Price'])
df1['Date'] = pd.to_datetime(df1['Date'])
df11['Date'] = pd.to_datetime(df11['Date'])
merged_df = pd.concat([df1, df11]).drop_duplicates(subset='Date').sort_values(by='Date')
merged_df.to_excel('xauusd.xlsx', index=False)